<a href="https://colab.research.google.com/github/byui-cse/cse480-notebooks/blob/master/10_3_Universal_Turing_Machines_and_Computer_Programs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universal Turing Machines and Computer Programs
## Supplemental Material on "The Big Picture"
### 12 March 2021

## Big Idea #1

There exist undecidable/uncomputable problems.

## Big Idea #2

There exist universal computers --- computers that can simulate any other computer or program.

### The Python ```exec``` function

executes strings of Python code:

In [ ]:
command = "print('abc', 5+2)"
exec(command)

We can use ```exec``` to write a **universal** Python program that executes other Python programs:

In [2]:
def universal(prog_string, in_string):
  # Execute the definition of the function in prog_string.
  # This defines the program, but doesn't invoke it.
  exec(prog_string)
  # Now that the function is defined, we can extract a reference to it.
  prog_function = extract_main_function(prog_string, locals())
  # Invoke the desired function with the desired input string.
  return prog_function(in_string)


Study these helper functions carefully. SISO means "String In String Out".

In [3]:
import re

def extract_main_function_name(prog_string):
   """Extract the name of the main function in a Python program.

    Args:

        prog_string (str): string containing the Python program to be
        examined.

    Returns:

        str: The name of the main function if one could be found,
           otherwise the empty string.

    """

   # This is the regular expression that searches for the main
   # function using the heuristic described above.
   main_function_regex = r'^def\s+([a-zA-Z0-9_]*)'
   match_result = re.search(main_function_regex, prog_string, re.MULTILINE )
   if match_result:
      return match_result.group(1)
   else:
      # Return empty string if we couldn't find any function
      # definitions. This should never happen when processing a
      # legitimate SISO program.
      return ''

def extract_main_function(prog_string, local_vars):
   """Given a Python program, return a reference
    to the "main" function: that is, the first
    Python function defined within prog_string.
    The local_vars parameter should be

    Args:

        prog_string (str): string containing the Python program to be
            examined.

        local_vars (dict): the "locals()" dictionary of the calling
            function, as explained further in the source code comment.

    Returns:

        fn: A reference to the main function if one could be
            found. Otherwise an Exception is raised.

    """

   function_name = extract_main_function_name(prog_string)
   # Python has a standard built-in dictionary called "locals()"
   # which contains, among other things, all the functions that are
   # currently defined. We can get a reference to the desired
   # function by looking it up in this dictionary, using the name of
   # the function as the key.
   if function_name in local_vars:
      prog_function = local_vars[function_name]
   else:
      raise Exception('function ' + function_name + \
                      ' not defined, so cannot extract or simulate it')
   return prog_function

def read_file(file_name):
   """Read a file, returning its contents as a single string.

    Args:

        file_name (str): The name of the file to be read.

    Returns:

        str: The contents of the file.
    """

   file_contents = ''
   with open(file_name) as input_file:
      file_contents = input_file.read()
   return file_contents

# Define a very short convenient alias for the read_file function
rf = read_file

def write_file(file_name, file_contents):
   """Write a file, overwriting any existing content with the given content.

    Args:

        file_name (str): The name of the file to be written or overwritten.

        file_contents (str): The contents of the file to be written,
            stored as a single string that may contain newlines.
    """
   with open(file_name, 'w') as output_file:
      output_file.write(file_contents)

def ESS(in_string1, in_string2):
   """Encode two strings as a single string.

    ESS is an acronym for Encode as Single String.  This function uses
    the encoding that consists of the length of the first string,
    followed by a space character, followed by the two strings
    concatenated together.

    Args:

        in_string1 (str): The first string to be encoded

        in_string2 (str): The second string to be encoded

    Returns:

        str: A single string encoding in_string1 and in_string2

    Example:

        >>> ESS('abc', 'defg')
        '3 abcdefg'
    """
   return str(len(in_string1)) + ' ' + in_string1 + in_string2

def DESS(in_string):
   """Decode a single string into two strings (inverse of ESS).

    DESS is an acronym for DEcode from Single String. This function converts a
    single string that encodes two strings back into the original two strings.
    DESS is the inverse of the function ESS.

    Args:

       in_string (str): The string to be decoded

    Returns:

       (str, str): A 2-tuple containing the two strings that were decoded from the input.

    Example:

       >>> DESS('3 abcdefg')
       ('abc', 'defg')

    """
   # split on the first space character
   (the_length, remainder) = in_string.split(' ', 1)
   in_string1 = remainder[:int(the_length)]
   in_string2 = remainder[int(the_length):]
   return (in_string1, in_string2)

### Some Examples

In [4]:
with open('containsGAGA.py', 'w') as f:
  f.write("""
def containsGAGA(in_string):
    if 'GAGA' in in_string:
        return 'yes'
    else:
        return 'no'
  """)

Verify the file is there:

In [ ]:
!cat containsGAGA.py

What do these output?

In [ ]:
universal(rf('containsGAGA.py'), 'GTTGAGA')

In [ ]:
universal(rf('containsGAGA.py'), 'GTTAA')

### Universal Turing Machines exist too

Here's a high-level argument:

1. Any Python program can be converted to a Turing machine.
2. We just saw the universal Python function (program -- if you save its code in a file 'universal.py'.)
3. Therefore 'universal.py' can be converted into an equivalent Turing machine, which is universal by definition (and can be made to receive only one parameter).

#### Universal Turing Machines

UTMs can be constructed explicitly --- here are two examples:

1. Alan Turing gave a suitable construction in his 1936 paper.
2. Marvin Minsky published a 7-state, 4-symbol UTM in the 1960s.

#### Universal computation occurs in the real world

|             |  simple                 | complex        |
|-------------|-------------------------|----------------|
| artificial  | Minsky's machine        | real computers |
|             | Rule 110 CA (see below) |                |
|             | Conway's Game of Life   |                |
|             | ...                     |                |
| natural     | ?                       | human brain    |

##### Rule 110 CA

The "Rule 110 Cellular Automaton" is a famous example of universal computation resulting from extremely simple rules:


In [9]:
# Conceptually, we think of the cells of the automaton being filled
# with zeros and ones, but we will encode these as '-' and 'G'
# respectively, since those characters happen to give a more appealing
# visual appearance when successive results are printed out.
zero = '-'
one = 'G'

# update_rule is a dictionary mapping a triple of cells to the new
# value of the middle cell of that triple in the next timestep.
update_rule = {
    (zero, zero, zero): zero,
    (zero, zero, one): one,
    (zero, one, zero): one,
    (zero, one, one): one,
    (one, zero, zero): zero,
    (one, zero, one): one,
    (one, one, zero): one,
    (one, one, one): zero,
    }

def rule_110(tape):
    """Implements a single timestep of the Rule 110 automaton.

    Args:

        tape (str): A string representing the current tape of zeros
            and ones.

    Returns:

        str: The updated contents of the tape after a single
            timestep. The left and right ends of the tape are assumed
            to be padded with zeros.

    """

    L = len(tape)
    updated_tape = []
    for i in range(L):
        # The left and right ends of the tape need to be treated
        # separately since they should be padded with a zero.
        if i == 0:
            # pad with a zero on the left
            (left, middle, right) = (zero, tape[i], tape[i+1])
        elif i == L-1:
            # pad with a zero on the right
            (left, middle, right) = (tape[i-1], tape[i], zero)
        else:
            # no padding required as we are in the interior of the tape
            (left, middle, right) = (tape[i-1], tape[i], tape[i+1])
        new_symbol = update_rule[(left, middle, right)]
        updated_tape.append(new_symbol)
    return ''.join(updated_tape)

def test_Rule_110():
    string_length = 64
    iterations = 40
    start_string = [zero for x in range(string_length)]
    one_locations = [string_length-25, string_length-1]
    # put a single one at each specified location
    for location in one_locations:
        start_string[location] = one

    tape = start_string
    for i in range(iterations):
        print(tape)
        tape = rule_110(tape)

    expected_tape = 'GGGGGG----G-GGG-GGGG---GGG---G-GGG--GG---G--GG-GGGGGG-----GG---G'
    assert tape == expected_tape

In [ ]:
test_Rule_110()

#### Using a universal program

With a universal program in hand, we can alter the effects of other programs in real time:

In [11]:
def repeat_CA_or_GA(in_string):
    if in_string == 'CA':
        return 'CACA'
    elif in_string == 'GA':
        return 'GAGA'
    else:
        return 'unknown'

In [12]:
def alter_GAGA_to_TATA(in_string):
  (prog_string, new_in_string) = DESS(in_string)
  val = universal(prog_string, new_in_string)
  if val == 'GAGA':
    return 'TATA'
  else:
    return val

#### Some decision problems are recognizable but undecidable

Recognizable means there's a program that 

1. **always terminates** with the right answer ("yes") on positive instances
2. Reliably is **never wrong** on negative instances
3. But it may **enter an infinite loop** on negative instances

###### Example

```yes_on_string``` and ```crash_on_string``` are recognizable but undecidable:

Here's a program that recognizes ```yes_on_string```:

In [14]:
def recognizes_yes_on_string(in_string):
  (prog_string, new_in_string) = DESS(in_string)
  val = universal(prog_string, new_in_string)
  if val == 'yes':
    return 'yes'
  else:
    return 'no'

In [15]:
def test_recognizes_yes_on_string():
  for (prog_name, in_string, solution) in \
      [('containsGAGA.py', 'GAGAGAGAG', 'yes'), \
       ('containsGAGA.py', 'TTTTGGCCGGT', 'no')]:
       combined_string = ESS(rf(prog_name), in_string)
       val = recognizes_yes_on_string(combined_string)
       print((prog_name, in_string), ':', val)
       assert val == solution

In [ ]:
test_recognizes_yes_on_string()